In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import udf

In [2]:
list=[['AA',[],[7],[7]],
      ['AA',[9],[4],[9]],
      ['AA',[8],[2],[8,2]],
      ['AA',[2],[3,4],[3,4]],
      ['AA',[1],[3,4],[2,3,4,5]],
     ['AA',[2,3],[2,3,4],[2,3,4,5]]]
df= spark.createDataFrame(list,['number','matriculate','name','model'])
df.show()

+------+-----------+---------+------------+
number|matriculate| name| model|
+------+-----------+---------+------------+
 AA| []| [7]| [7]|
 AA| [9]| [4]| [9]|
 AA| [8]| [2]| [8, 2]|
 AA| [2]| [3, 4]| [3, 4]|
 AA| [1]| [3, 4]|[2, 3, 4, 5]|
 AA| [2, 3]|[2, 3, 4]|[2, 3, 4, 5]|
+------+-----------+---------+------------+

In [3]:
'''expression= """filter(model, x->  x in name)"""
df.withColumn("name2", F.expr(expression)).show()'''

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling z:org.apache.spark.sql.functions.expr.
: org.apache.spark.sql.catalyst.parser.ParseException: 
extraneous input 'in' expecting {')', ','}(line 1, pos 21)

== SQL ==
filter(model, x-> x in name)
---------------------^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:241)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:117)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:53)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parseExpression(ParseDriver.scala:44)
	at com.databricks.sql.parser.DatabricksSqlParser.parseExpression(DatabricksSqlParser.scala:46)
	at org.apache.spark.sql.functions$.expr(functions.scala:1366)
	at org.apache.spark.sql.functions.expr(functions.scala)
	at sun.reflect.GeneratedMethodAccessor465.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 ParseException Traceback (most recent call last)
 <command-3387556583879083> in <module> 
 1 expression = """filter(model, x-> x in name)""" 
 ----> 2 df . withColumn ( "name2" , F . expr ( expression ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/functions.py in expr (str) 
 673 """
 674 sc = SparkContext . _active_spark_context
 --> 675 return Column ( sc . _jvm . functions . expr ( str ) ) 
 676 
 677 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 72 if s . startswith ( 'org.apache.spark.sql.catalyst.parser.ParseException: ' ) : 
 ---> 73 raise ParseException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 74 if s . startswith ( 'org.apache.spark.sql.streaming.StreamingQueryException: ' ) : 
 75 raise StreamingQueryException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 ParseException : "\nextraneous input 'in' expecting {')', ','}(line 1, pos 21)\n\n== SQL ==\nfilter(model, x-> x in name)\n---------------------^^^\n"

In [4]:
'''ar1=[3,4,4]
ar2=[2,3,4]
def intersect(ar1,ar2):
     return [i for i in ar1 if i in ar2]
  
udf1= F.udf(intersect, ArrayType(LongType()))'''




In [5]:
df.withColumn("nm", F.array_intersect("name","model")).withColumn("nm1", F.array_intersect("matriculate","model")).withColumn("flag2", F.when((F.col("nm1")==F.col("matriculate"))&(F.size("matriculate")!=0),F.lit(False)).otherwise(F.lit(True))).withColumn("flag", F.when((F.col("flag2")==F.lit(True))&(F.col("nm")!=F.col("name")), F.lit(False)).otherwise(F.col("flag2")))\
.show()

+------+-----------+---------+------------+---------+------+-----+-----+
number|matriculate| name| model| nm| nm1|flag2| flag|
+------+-----------+---------+------------+---------+------+-----+-----+
 AA| []| [7]| [7]| [7]| []| true| true|
 AA| [9]| [4]| [9]| []| [9]|false|false|
 AA| [8]| [2]| [8, 2]| [2]| [8]|false|false|
 AA| [2]| [3, 4]| [3, 4]| [3, 4]| []| true| true|
 AA| [1]| [3, 4]|[2, 3, 4, 5]| [3, 4]| []| true| true|
 AA| [2, 3]|[2, 3, 4]|[2, 3, 4, 5]|[2, 3, 4]|[2, 3]|false|false|
+------+-----------+---------+------------+---------+------+-----+-----+

In [6]:
if model column contain all values of name columns and not contain matricule array ==> Flag = True
else false.

In case if model contain matricule and contain name (like in line 3 in the dataframe example) ==> Flag = False

File "<command-1250592606695349>" , line 1 
 if model column contain all values of name columns and not contain matricule array ==> Flag = True 
 ^
 SyntaxError : invalid syntax